# Download all the Python Libraries

In [20]:
# Check if the copmuter is on google colab
import sys
if 'google.colab' in sys.modules:
    print("Running on Google Colab")
    !pip install rich
    !pip install -q -U bitsandbytes
    !pip install -q -U git+https://github.com/huggingface/transformers.git 
    !pip install -q -U git+https://github.com/huggingface/peft.git
    !pip install -q -U git+https://github.com/huggingface/accelerate.git
else:
    print("Not running on Google Colab")
from rich import print
import logging
from pathlib import Path
logger = logging.getLogger(__name__)
#ROOT_PATH = Path(__file__).parent.parent

Not running on Google Colab

# Check the GPU env
1. You can check the GPU in the Google Colab by clicking  and efficieny
2. Check if the GPU can use bfloat16 most effective as most model are pre-trained with bfloat16

In [21]:
import torch
from rich import print
if torch.cuda.is_available():
    !nvidia-smi
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print("Cuda capability: ", torch.cuda.get_device_capability(0))
    '''
    On pre-ampere hardware bf16 works, but doesn't provide speed-ups compared to fp32 matmul operations, and some matmul operations are failing outright, so this check is more like "guaranteed to work and be performant" than "works somehow".  https://github.com/pytorch/pytorch/issues/75427
    '''
    print(f"bfloat16 support: { torch.cuda.is_bf16_supported()}") 

# Set the Seed Environment of the Notebook to ensure the reproducibility

In [22]:
from transformers import set_seed

DEFAULT_SEED = 42

set_seed( DEFAULT_SEED )

# Constant Variable

In [23]:
MAX_LENGTH  = 1024
BITS  = 4
LR = 1e-5
EPOCHS = 2
# Whether to use bf16 (preferred on A100's)."

# Set Up Local Training Root

# Download the Datset from the Hugging Face Datset Face Dataset

In [24]:
DEFAULT_TRAINING_DATASET = "Rami/prompts"
from datasets import load_dataset
training_dataset = load_dataset(
    "Rami/prompts",
)
print(
    training_dataset
)


Found cached dataset parquet (/home/null/.cache/huggingface/datasets/Rami___parquet/Rami--prompts-c0e384db99c3a811/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'response', 'views', 'instruction', 'category'],
        num_rows: 341
    })
})

# Download the Tokenizers
1. We are suing Dolly model which was trained on the Pythia model. Instead we are recreating the dollvy tokenizer from the Pythia tokenizer

In [25]:
from transformers import AutoTokenizer

# Special Tokens
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Input:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
RESPONSE_KEY_NL = f"{RESPONSE_KEY}\n"
DEFAULT_SEED = 42

PRETRAINED_MODEL_NAME_OR_PATH = "databricks/dolly-v2-3b"
eleutherai_python_3b = "EleutherAI/pythia-2.8b"
dolly_v2_tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME_OR_PATH)
print(dolly_v2_tokenizer)
pythia_tokenizer = AutoTokenizer.from_pretrained(eleutherai_python_3b)
print(pythia_tokenizer)

# Make sure that the pad token is the end of the tokens
pythia_tokenizer.pad_token = pythia_tokenizer.eos_token

# Add special tokens for End , Instruction , Response Key

pythia_tokenizer.add_special_tokens({
    "additional_special_tokens": [
        END_KEY,
        INSTRUCTION_KEY,
        RESPONSE_KEY,
    ]
})

print(pythia_tokenizer)

GPTNeoXTokenizerFast(name_or_path='databricks/dolly-v2-3b', vocab_size=50254, 
model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', 
special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 
'pad_token': '<|endoftext|>', 'additional_special_tokens': ['### End', '### Instruction:', '### Response:']}, 
clean_up_tokenization_spaces=True)

GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-2.8b', vocab_size=50254, 
model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', 
special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, 
clean_up_tokenization_spaces=True)

GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-2.8b', vocab_size=50254, 
model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', 
special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 
'pad_token': '<|endoftext|>', 'additional_special_tokens': ['### End', '### Instruction:', '### Response:']}, 
clean_up_tokenization_spaces=True)

# Process the dataset
1. Convert the dataset into instruction

## Instruct Format String and Constants

In [26]:
INTRO_BLURB = (
    "Below is an instruction that describes a task. Write a response that appropriately completes the request."
)
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Input:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
RESPONSE_KEY_NL = f"{RESPONSE_KEY}\n"
DEFAULT_SEED = 42
# This is a training prompt that does not contain an input string.  The instruction by itself has enough information
# to respond.  For example, the instruction might ask for the year a historic figure was born.
PROMPT_NO_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}
{response}

{end_key}""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
    response="{response}",
    end_key=END_KEY,
)

# This is a training prompt that contains an input string that serves as context for the instruction.  For example,
# the input might be a passage from Wikipedia and the intruction is to extract some information from it.
PROMPT_WITH_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{input_key}
{input}

{response_key}
{response}

{end_key}""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    input_key=INPUT_KEY,
    input="{input}",
    response_key=RESPONSE_KEY,
    response="{response}",
    end_key=END_KEY,
)

# This is the prompt that is used for generating responses using an already trained model.  It ends with the response
# key, where the job of the model is to provide the completion that follows it (i.e. the response itself).
PROMPT_FOR_GENERATION_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)

In [27]:
from datasets import Dataset, load_dataset

def load_training_dataset(path_or_dataset: str = DEFAULT_TRAINING_DATASET) -> Dataset:
    logger.info(f"Loading dataset from {path_or_dataset}")
    dataset = load_dataset(path_or_dataset)["train"]
    logger.info("Found %d rows", dataset.num_rows)

    def _add_text(rec):
        instruction = rec["instruction"]
        response = rec["response"]
        context = rec.get("context")

        if not instruction:
            raise ValueError(f"Expected an instruction in: {rec}")

        if not response:
            raise ValueError(f"Expected a response in: {rec}")

        # For some instructions there is an input that goes along with the instruction, providing context for the
        # instruction.  For example, the input might be a passage from Wikipedia and the instruction says to extract
        # some piece of information from it.  The response is that information to extract.  In other cases there is
        # no input.  For example, the instruction might be open QA such as asking what year some historic figure was
        # born.
        if context:
            rec["text"] = PROMPT_WITH_INPUT_FORMAT.format(instruction=instruction, response=response, input=context)
        else:
            rec["text"] = PROMPT_NO_INPUT_FORMAT.format(instruction=instruction, response=response)
        return rec

    dataset = dataset.map(_add_text)

    return dataset

## Preprocess the dataset using the tokenizers

In [28]:
import logging
from functools import partial
from pathlib import Path
from typing import Any, Dict, List, Tuple, Union
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    PreTrainedTokenizer,
    Trainer,
    TrainingArguments,
    set_seed,
)
def preprocess_batch(batch: Dict[str, List], tokenizer: AutoTokenizer, max_length: int) -> dict:
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed=DEFAULT_SEED, training_dataset: str = DEFAULT_TRAINING_DATASET) -> Dataset:
    """Loads the training dataset and tokenizes it so it is ready for training.

    Args:
        tokenizer (AutoTokenizer): Tokenizer tied to the model.
        max_length (int): Maximum number of tokens to emit from tokenizer.

    Returns:
        Dataset: HuggingFace dataset
    """

    dataset = load_training_dataset(training_dataset)

    logger.info("Preprocessing dataset")
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["instruction", "context", "response", "text", "category"],
    )

    # Make sure we don't have any truncated records, as this would mean the end keyword is missing.
    logger.info("Processed dataset has %d rows", dataset.num_rows)
    dataset = dataset.filter(lambda rec: len(rec["input_ids"]) < max_length)
    logger.info("Processed dataset has %d rows after filtering for truncated records", dataset.num_rows)

    logger.info("Shuffling dataset")
    dataset = dataset.shuffle(seed=seed)

    logger.info("Done preprocessing")

    return dataset

In [29]:
processed_dataset = preprocess_dataset(
    tokenizer = pythia_tokenizer,
    max_length = MAX_LENGTH,
    seed = DEFAULT_SEED,
)

# Split the Dataset

split_dataset = processed_dataset.train_test_split(test_size = 0.2, seed=DEFAULT_SEED, shuffle = True)

train_datset = split_dataset["train"]
val_dataset = split_dataset["test"]

logger.info("Train data size: %d", split_dataset["train"].num_rows)
logger.info("Test data size: %d", split_dataset["test"].num_rows)

Found cached dataset parquet (/home/null/.cache/huggingface/datasets/Rami___parquet/Rami--prompts-c0e384db99c3a811/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/341 [00:00<?, ? examples/s]

Map:   0%|          | 0/341 [00:00<?, ? examples/s]

Filter:   0%|          | 0/341 [00:00<?, ? examples/s]

## Data Collator For Completion Only LM

In [30]:
import numpy as np
class DataCollatorForCompletionOnlyLM(DataCollatorForLanguageModeling):
    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        batch = super().torch_call(examples)

        # The prompt ends with the response key plus a newline.  We encode this and then try to find it in the
        # sequence of tokens.  This should just be a single token.
        response_token_ids = self.tokenizer.encode(RESPONSE_KEY_NL)

        labels = batch["labels"].clone()

        for i in range(len(examples)):

            response_token_ids_start_idx = None
            for idx in np.where(batch["labels"][i] == response_token_ids[0])[0]:
                response_token_ids_start_idx = idx
                break

            if response_token_ids_start_idx is None:
                raise RuntimeError(
                    f'Could not find response key {response_token_ids} in token IDs {batch["labels"][i]}'
                )

            response_token_ids_end_idx = response_token_ids_start_idx + 1

            # Make pytorch loss function ignore all tokens up through the end of the response key
            labels[i, :response_token_ids_end_idx] = -100

        batch["labels"] = labels

        return batch

In [31]:
if torch.cuda.get_device_name(0) in ["A100-SXM4-40GB", "A100-SXM4-80GB"]:
        torch.backends.cuda.matmul.allow_tf32 = True 
# https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html
data_collator = DataCollatorForCompletionOnlyLM(
    tokenizer = pythia_tokenizer,
    mlm=False, 
    return_tensors="pt", 
    pad_to_multiple_of =   64 if torch.cuda.get_device_name(0) in ["A100-SXM4-40GB", "A100-SXM4-80GB"] else 8
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_387553/2571383233.py:1 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_387553/2571383233.py'                       │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/torch/cuda/__init__.py:365 in  │
│ get_device_name                                                                                  │
│                                                                                                  │
│    362 │   Returns:                                                                              │
│    363 │   │   str: the name of the device                                                       │
│    364 │   """                                                                                   │
│ ❱  365 │   return get_device_properties(device).name                                             │
│    366                                                                                           │
│    367                                                                                           │
│    368 def get_device_capability(device: Optional[_device_t] = None) -> Tuple[int, int]:         │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/torch/cuda/__init__.py:395 in  │
│ get_device_properties                                                                            │
│                                                                                                  │
│    392 │   Returns:                                                                              │
│    393 │   │   _CudaDeviceProperties: the properties of the device                               │
│    394 │   """                                                                                   │
│ ❱  395 │   _lazy_init()  # will define _get_device_properties                                    │
│    396 │   device = _get_device_index(device, optional=True)                                     │
│    397 │   if device < 0 or device >= device_count():                                            │
│    398 │   │   raise AssertionError("Invalid device id")                                         │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/torch/cuda/__init__.py:247 in  │
│ _lazy_init                                                                                       │
│                                                                                                  │
│    244 │   │   # are found or any other error occurs                                             │
│    245 │   │   if 'CUDA_MODULE_LOADING' not in os.environ:                                       │
│    246 │   │   │   os.environ['CUDA_MODULE_LOADING'] = 'LAZY'                                    │
│ ❱  247 │   │   torch._C._cuda_init()                                                             │
│    248 │   │   # Some of the queued calls may reentrantly call _lazy_init();                     │
│    249 │   │   # we need to just return without initializing in that case.                       │
│    250 │   │   # However, we must not let any *other* threads in!                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a 
driver from http://www.nvidia.com/Download/index.as

# Download the Model
1. Torch Datat

## Setup Bits and Butes Config

## 4 Bit Configuration

In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/gpt-neox-20b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_threshold = 6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


## Download the LM Models
Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [33]:
from transformers import AutoModelForCausalLM
assert torch.cuda.is_available(), "You need to have a GPU to run this notebook."
n_gpus = torch.cuda.device_count()

free_in_GB = int(torch.cuda.mem_get_info()[0]/1024**3)
max_memory = f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'

n_gpus = torch.cuda.device_count()
max_memory = {i: max_memory for i in range(n_gpus)}
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path = PRETRAINED_MODEL_NAME_OR_PATH,
    trust_remote_code = True,
    use_cache = False,
    torch_dtype = torch.bfloat16,
    device_map = "auto",
    load_in_8bit = False,
    low_cpu_mem_usage = True,
    max_memory =  max_memory,
    quantization_config = bnb_config,
)


print(model)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_387553/3038055455.py:2 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_387553/3038055455.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AssertionError: You need to have a GPU to run this notebook.

In [34]:
from peft import prepare_model_for_kbit_training
def print_trainable_parameters(args, model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    if args.bits == 4: trainable_params /= 2
    print(
        f"trainable params: {trainable_params} || "
        f"all params: {all_param} || "
        f"trainable: {100 * trainable_params / all_param}"
    )

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_387553/2799831422.py:19 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_387553/2799831422.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

## Set up the LoRa Models

In [35]:
from peft import LoraConfig, get_peft_model
import bitsandbytes as bnb
def find_all_linear_names( model):
    cls = bnb.nn.Linear4bit if BITS == 4 else (bnb.nn.Linear8bitLt if BITS == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])


    if 'lm_head' in lora_module_names: # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules = find_all_linear_names(  model),
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_387553/701706016.py:19 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_387553/701706016.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

# Training Arguments 
1. WarmUp Steps 
2. Learning Rate 
   1.  0.0001 <path_or_name> https://github.com/artidoro/qlora/tree/main 

In [36]:
from transformers import Trainer, TrainingArguments
batch_size: int = 128
micro_batch_size: int = 4
gradient_accumulation_steps = batch_size // micro_batch_size
learning_rate: float = 3e-4, # 
epochs = 16
print(f"learning_rate: {learning_rate}")
trainer_arguments = TrainingArguments(
            output_dir = "outputs",
            per_device_train_batch_size = batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            warmup_steps=100,
            num_train_epochs = epochs,
            learning_rate=learning_rate,
            bf16=True,
            logging_steps=10,
            optim = "paged_adamw_8bits",
            save_strategy="steps",
)

learning_rate: (0.0003,)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_387553/3948449762.py:8 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_387553/3948449762.py'                       │
│ <string>:111 in __init__                                                                         │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/transformers/training_args.py: │
│ 1315 in __post_init__                                                                            │
│                                                                                                  │
│   1312 │   │   │   if not is_torch_available():                                                  │
│   1313 │   │   │   │   raise ValueError("lr_scheduler_type reduce_lr_on_plateau requires torch>  │
│   1314 │   │                                                                                     │
│ ❱ 1315 │   │   self.optim = OptimizerNames(self.optim)                                           │
│   1316 │   │   if self.adafactor:                                                                │
│   1317 │   │   │   warnings.warn(                                                                │
│   1318 │   │   │   │   "`--adafactor` is deprecated and will be removed in version 5 of 🤗 Tran  │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/enum.py:385 in __call__                      │
│                                                                                                  │
│    382 │   │   `type`, if set, will be mixed in as the first base class.                         │
│    383 │   │   """                                                                               │
│    384 │   │   if names is None:  # simple value lookup                                          │
│ ❱  385 │   │   │   return cls.__new__(cls, value)                                                │
│    386 │   │   # otherwise, functional API: we're creating a new Enum type                       │
│    387 │   │   return cls._create_(                                                              │
│    388 │   │   │   │   value,                                                                    │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/enum.py:718 in __new__                       │
│                                                                                                  │
│    715 │   │   │   │   │   │   │   )                                                             │
│    716 │   │   │   │   if not isinstance(exc, ValueError):                                       │
│    717 │   │   │   │   │   exc.__context__ = ve_exc                                              │
│ ❱  718 │   │   │   │   raise exc                                                                 │
│    719 │   │   finally:                                                                          │
│    720 │   │   │   # ensure all variables that could hold an exception are destroyed             │
│    721 │   │   │   exc = None                                                                    │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/enum.py:700 in __new__                       │
│                                                                                                  │
│    697 │   │   # still not found -- try _missing_ hook      

In [37]:
trainer = Trainer(
    model=model,
    tokenizer = pythia_tokenizer,
    args = trainer_arguments,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    data_collator=data_collator,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_387553/2539363348.py:2 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_387553/2539363348.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

# Start training

In [ ]:
logger.info("Training")
trainer.train()

# Push the Model to the Hugging Face Model Hub

In [ ]:
model.push_to_hub("Rami/prompt_generator")

# References
[1] [Dolly Github](https://github.com/databrickslabs/dolly/blob/5021d941d95dddcf1f00d978d7f944709873f419/training/trainer.py#L138)
[2] https://gist.github.com/Birch-san/57878c4a27cf34f57d3e861865a7d0a2
[3] https://github.com/artidoro/qlora/blob/main/qlora.py 
[4] https://github.com/tloen/alpaca-lora/blob/main/finetune.py 